In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dt = 1e-12 #s

Classe das partículas

In [22]:
class Particula:
    def __init__(self, raio: float, massa: float, posicao: float, velocidade: float) -> None:
        self.massa = massa
        self.raio = raio
        
        self.posicao_x = posicao[0]
        self.posicao_y = posicao[1]

        self.velocidade_x = velocidade[0]
        self.velocidade_y = velocidade[1]

        self.momento_x = massa * velocidade[0]
        self.momento_y = massa * velocidade[1]

Testagem

In [28]:
p1 = Particula(1, 10, [0,0], [1,2])